![title](./media/walmart_christmas.jpg)

# Walmart: Weihnachtsgeschäft Just in Time

Modulprojekt für die Vorlesung __Data-Mining-Process__

_Dozent:_ Prof. Dr. Johannes Maucher  
_Studenten:_ Stephanie Flohr, Benedikt Haußner, Robert Masendorf

Für unser Projekt analysiseren wir Verkaufsdaten der amerikanischen Supermarktkette _Walmart_.

## BUSINESS UNDERSTANDING: Ursprung und grober Inhalt des Datensatzes

### Was ist Walmart?
Walmart ist der wertvollste Einzelhändler der Welt mit einer Marktkapitalisierung von 246,2 Mrd. US-Dollar. Weitere Fakten:
+ Anzahl Filialen weltweit: ca 11.600
+ Jahresumsatz 2017: 50,3 Mrd. US-Dollar
+ Gewinn 2017: 9,9 Mrd. US-Dollar
+ Anzahl der Mitarbeiter weltweit: ca 2 Millionen

Quelle: [Wikipedia](https://de.wikipedia.org/wiki/Walmart)

### _Just in Time_ bei Walmart
[Wikipedia](https://de.wikipedia.org/wiki/Walmart)

Walmart´s Strategie beruht auf ein Konzept aus geringen Preisen und gewinnmargen begleitet durch geringe Lohn- und Gehaltzahlungen trotz hohem Profit. Ein wichtiger Baustein in diesem Konzept ist auch die Produktion nach Just in Time. Walmart unterhält keine Warenlager, sondern entwickelte eine ausgefeilte Logistik, durch die die Produkte direkt aus der Produktion zu den Logistzentren geliefert werden. Dadurch sind keine Zwischenlager mehr notwendig. Um dieses Konzept weltweit umsetzen zu können, beschäftigt Walmart 2000 Analysten. 

### Wettbewerb auf Kaggle
__TODO:__ Hier eine Kurze Beschreibung einfügen mit Link zum Wettbewerb: [Walmart - Store Sales Forecasting](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting).
 
+ was enthält der datensatz
+ wir machen NICHT die Competition

### Ziel unserer Arbeit
__TODO:__ Ausführlicher Beschreiben Es ist unser Ziel, aus Daten der Jahre 2010 und 2011 eine Prognose für die Verkaufszahlen des Jahres 2011 zu ermitteln.

## DATA UNDERSTANDING: Was wissen wir und was wollen wir wissen?

### Inhalte der Quell-Dateien

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

Das von Walmart zur Verfügung gestellte Datenpaket beinhaltet fünf csv-Datein.
+ features.csv
+ sampleSubmission.csv
+ stores.csv
+ test.csv
+ train.csv

Im ersten Schritt laden wir die Datein als Data Frames in das Notebook und unterschuchen die gegebenen Informationen auf ihre Gebrauchbarkeit.

Weitere Informationen zu den einzelen Datasets: [Walmart - Data](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data)

##### train.csv

In [2]:
data = pd.read_csv('./data/train.csv')
trainDf = pd.DataFrame(data)
display(trainDf.head())
#trainDf.dtypes

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


#### Attribute:
Hierbei handelt es sich um das historical train set. Der beinhaltete Zeitraum geht von 2010-02-05 bis 2012-11-01. Die Competitionteilnehmer sollten mit Hilfe deses Sets arbeiten.
+ Store: Bezeichnet einen Store mit einer Nummer, um eine Zuordnung herzustellen
+ Dept: Stores sind in unterschiedliche Bereiche aufgeteilt. Diese Aufteilung in sogenannte Departments ist mit einer Nummer aufgezeigt. 
+ Date: Datum des Tages, an dem der wöchentliche Verkauf festgehalten worden ist.
+ Weekly_Sales: Wöchentliche Verkaufszahlen
+ IsHoliday: Beschreibt, ob es in der Woche Feiertage oder Ferien gab.

#### test.csv

In [3]:
data = pd.read_csv('./data/test.csv')
testDf = pd.DataFrame(data)
display(testDf.head())
#testDf.dtypes

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


#### Attribute:
Es handelt sich hierbei um dieselben Attribute, wie im train set. Nur das `Weekly_Sales`fehlt. Das Fiele ist in der Orginal competition als Testfile erstellt worden. Die Teilnehmer des Wettbewerbs sollten die weekly sales für den im Test festgelegten Zeitraum vorhersagen.

#### features.csv

In [4]:
data = pd.read_csv('./data/features.csv')
featuresDf = pd.DataFrame(data)
display(featuresDf.head())
#featuresDf.dtypes

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


#### Attribute: 
Das features.csv enthält zusätzliche Attribute, die in die weekly_sales mit Einfluss genommen haben. Auf dem ersten Blick ist zu erkennen, dass die Markdowns sehr wenig Informationen beinhalten. Im Kapitel "Univariate Analysis" wird auf diesen Aspekt genauer eingegangen.

MarkDown beschreibt anonymisierte Daten zu Preisnachlässen. Da nur bis November 2011 Daten hierfür verfügbar sind, sind sehr viele Angaben dazu mit NaN hinterlegt. 

#### stores.csv

In [5]:
data = pd.read_csv('./data/stores.csv')
storesDf = pd.DataFrame(data)
display(storesDf.head())
#storesDf.dtypes

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


Dieses File beinhaltet anonymisierte Informationen zu den einzelnen Stores.

#### Auswertung der Attribute und Bewertung der zu nutzenden Daten

### Univariate Analyse

__TODO:__ 
+ für die relevanten attribute die Verteilung ect anschauen
+ Qulitätsanalyse: fehlen Werte ect?
+ auf jeden Fall eine hübsche Grafik mit Bokeh!

In [6]:
trainDf.isnull().sum()

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

In [7]:
testDf.isnull().sum()

Store        0
Dept         0
Date         0
IsHoliday    0
dtype: int64

In [8]:
featuresDf.isnull().sum()

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

#### Auswertung:
Der Zeitraum, in dem Markdowns erfasst worden sind, umfasst nicht alle Datenpunkte. Daher `featureDF` enthält viele leere Informationen. Die MarkDowns helfen uns an dieser Stelle nicht weiter. Mit `.dropna()` werden diese leeren Informationen aus dem Dataframe herausgeschnitten.

In [15]:
featuresDf = featuresDf.dropna()
featuresDf.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
92,1,2011-11-11,59.11,3.297,10382.90,6115.67,215.07,2406.62,6551.42,217.998085,7.866,False
93,1,2011-11-18,62.25,3.308,6074.12,254.39,51.98,427.39,5988.57,218.220509,7.866,False
94,1,2011-11-25,60.14,3.236,410.31,98.00,55805.51,8.00,554.92,218.467621,7.866,True
95,1,2011-12-02,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866,False
96,1,2011-12-09,43.93,3.158,4640.65,19.00,105.02,3639.42,14461.82,218.961846,7.866,False


In [14]:
featuresDf.isnull().sum()

Store           0
Date            0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday       0
dtype: int64

In [24]:
featuresDf.describe()

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,2069.000000,2069.000000,2069.000000,2069.000000,2069.000000,2069.000000,2069.000000,2069.000000,2069.000000,2069.000000
mean,20.386660,52.516979,3.600298,9717.665297,4426.859778,2247.535911,4213.162687,5056.916061,175.587370,7.252611
std,12.076174,18.483053,0.279335,11377.636869,10154.658061,13305.985612,8359.765985,17807.168442,39.963914,1.684774
min,1.000000,-7.290000,2.872000,-563.900000,-265.760000,-179.260000,0.460000,-185.170000,129.816710,3.684000
25%,10.000000,38.100000,3.409000,3542.800000,67.990000,5.840000,404.050000,2014.840000,137.423897,6.162000
50%,20.000000,51.420000,3.610000,6397.720000,404.610000,34.100000,1401.230000,3470.490000,189.707605,7.191000
75%,29.000000,66.610000,3.796000,11265.620000,3900.310000,137.990000,3834.440000,5674.110000,219.970560,8.256000
max,45.000000,95.910000,4.301000,103184.980000,104519.540000,149483.310000,67474.850000,771448.100000,228.976456,12.890000


In [9]:
storesDf.isnull().sum()

Store    0
Type     0
Size     0
dtype: int64

### Entscheidung: 
Wir wollen für unsere Arbeit mit dem Trainingsdataset arbeiten. Es enthält vorerst alle wichtigen Informationen und ist geeignet, um durch weitere Daten erweitert zu werden. 

### Analyse: wollen wir evtl noch weitere externe Daten einbeziehen?

__TODO:__ 
+ eine andere Quelle wäre evtl ganz cool, aber nur, wenn uns was leichtes einfällt

## DATA PREPARATION: was wollen wir verwenden und wie?

### inhaltlicher Zuschnitt der Daten aufs Projektziel

__TODO:__ 
+ Feature Selection
+ Feature Enigneering

__TODO:__ 
+ kurze Erklärung, was in diesem Kapitel passieren soll

### zeitlicher Zuschnitt der Daten auf das Projektziel

__TODO:__ 
+ warum wir nicht einfach mit allen Daten arbeiten
+ weshalb wir die Monate ausgewählt haben, die wir ausgewählt haben

__TODO:__ 
+ Begründung, warum wir manuell splitten

In [ ]:
dataDf.Date = pd.to_datetime(dataDf.Date)
dataDf.head()
dataDf.dtypes

In [ ]:
train_df = dataDf[((dataDf.Date >= '2010-09-01') & (dataDf.Date <= '2010-12-31')) | ((dataDf.Date >= '2011-09-01') & (dataDf.Date <= '2011-11-30'))]
train_df.reset_index(inplace=True, drop=True)

In [ ]:
train_df.loc[(train_df['Store'] == 1) & (train_df['Dept'] ==1)]

In [ ]:
test_df = dataDf[(dataDf.Date >= '2011-12-01') & (dataDf.Date <= '2011-12-31')]
test_df.reset_index(inplace=True, drop=True)

In [ ]:
test_df.loc[(test_df['Store'] == 1) & (test_df['Dept'] ==1)]

In [ ]:
train_df.plot(x='Date', y='Weekly_Sales', figsize=(15,10), secondary_y='Dept')

In [ ]:
train_df.groupby('Date').agg(['sum']).reset_index().plot(x='Date', y='Weekly_Sales',kind="bar", figsize=(12,7)).set_ylabel("Weekly Sales in 10 mio.")
plt.show()

__TODO:__ 
+ Ergebnisbeschreibung, warum wir denken, hier evtl weiterzukommen

### hier auf Sliding Window eingehen?

__TODO:__ 
+ irgendwo müssen wir erklären, was das ist und wie es geht. 
+ am besten evtl hier, so hat man nicht am anfang nur theorie. 
+ und dann gleich mit python umsetzen. 
+ ein tutorial (weiß nciht, obs was taugt): [AnalyticsVidhya](https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/)

## MODELING

__TODO:__ 
+ Hier wollen wir sowohl mit Baum, als auch mit Netz arbeiten. 
+ Wir sollten mit der Grafik von scikit-learn begründen, warum wir welche Technik nutzen [hatten wir schonmal](http://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)

## EVALUATION

__TODO:__ 
+ was sind hier gute maßzahlen?

## DEPLOYMENT

__TODO:__ 
+ brauchen wir den schritt überhaupt?
+ evtl stattdessen einfach: __LESSONS LEARNED__ und __IDEEN FÜR DIE ZUKUNFT__ machen oder so